<h1 align="center"> Language Classification using Naive Bayes </h1>
<h4 align="center"> ~ Shivam Shukla </h4>


<h6 align="center"> Given a sentence in any of following language, this model will try to predict the language.</h6>

## [Slovak Text](https://sk.wikipedia.org/wiki/Jazveč%C3%ADk)
Mnohí ľudia, ktorí vidia na ulici jazvečíka s podlhovastým telom vôbec nevedia o tom, že tento malý štvornohý a veľmi obľúbený spoločník je pri dobrom výcviku obratným, vynikajúcim a spoľahlivým poľovným psom. Ako poľovný pes je mnohostranne využiteľný, okrem iného ako durič na brlohárenie. Králičí jazvečík sa dokáže obratne pohybovať v králičej nore. S inými psami a deťmi si nie vždy rozumie.

## [Czech Text](https://cs.wikipedia.org/wiki/Jezevč%C3%ADk)
Úplně první zmínky o psech podobných dnešním jezevčíkům nacházíme až ve Starém Egyptě, kde jsou vyobrazeni na soškách a rytinách krátkonozí psi s dlouhým hřbetem a krátkou srstí. Jednalo se ale o neustálený typ bez ustáleného jména. Další zmínky o jezevčících nacházíme až ve 14 - 15. století. Jedná se o psa, který se nejvíce podobá dnešnímu typu hladkosrstého standardního jezevčíka.


## [English Text](https://en.wikipedia.org/wiki/Dachshund)
While classified in the hound group or scent hound group in the United States and Great Britain, the breed has its own group in the countries which belong to the Fédération Cynologique Internationale (World Canine Federation). Many dachshunds, especially the wire-haired subtype, may exhibit behavior and appearance that are similar to that of the terrier group of dogs.

In [2]:
import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import numpy as np
import string

from collections import defaultdict

from sklearn.metrics import f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

import joblib
import pickle as pkl

from helper_code import *

## Data Exploration and Visualization

In [3]:
def read_file(filename):
    with open(filename,'r') as f:
        data = f.readlines()
    return data

In [4]:
raw_data = dict()

In [6]:
raw_data['sk'] = read_file('Data/Sentences/train_sentences.sk')
raw_data['en'] = read_file('Data/Sentences/train_sentences.en')
raw_data['ch'] = read_file('Data/Sentences/train_sentences.cs')

In [31]:
s=0
for i,j in raw_data.items():
    print(i)
        
    print(s+lent for sentence in j ])    

sk
[['Pán', 'de', 'Grandes', 'Pascual', 'jasne', 'vysvetlil,', 'aká', 'je', 'pridaná', 'hodnota', 'definície', 'terorizmu,', 'vyjadrením,', 'že', 'definícia,', 'ktorú', 'sme', 'používali', 'predtým,', 'bola', 'dôležitejšia,', 'ale', 'aj', 'tým,', 'že', 'trval', 'na', 'zozname', 'priestupkov.'], ['Keby', 'sme', 'sa', 'nepokúsili', 'o', 'nemožné,', 'ako', 'by', 'sme', 'mohli', 'dosiahnuť', 'zmenu?'], ['Oznámil,', 'že', 'budúci', 'týždeň', 'sa', 'zúčastní', 'schôdze', 'Výboru', 'pre', 'zahraničné', 'veci.'], ['Niekedy', 'by', 'sa', 'mohlo', 'zdať,', 'že', 'ide', 'o', 'hru', 'alebo', 'pokrytectvo.'], ['Jej', 'účelom', 'je', 'tiež', 'identifikovať', 'akékoľvek', 'slabé', 'miesta', 'v', 'systéme', 'a', 'právne', 'nedostatky', 'súdnej', 'spolupráce', 'v', 'trestných', 'veciach,', 'aby', 'sa', 'európskym', 'zákonodarcom', 'poskytli', 'všetky', 'informácie', 'potrebné', 'na', 'správne', 'politické', 'a', 'regulačné', 'hodnotenie.'], ['Na', 'dosiahnutie', 'tohto', 'cieľa', 'je', 'mimoriadne', 'd

In [21]:
def show_statistics(data):
    for language, sentences in data.items():
        
        number_of_sentences = len(sentences)
        number_of_words = len([sentence.split(' ') for sentence in sentences])
        number_of_unique_words = len([sentence.split(' ') for sentence in sentences])
        sample_extract = ''
        
        # take a few minutes to try populate these variables
        
        # here is a hint -- word_list breaks the collections of sentences into a list of words
        word_list = ' '.join(sentences).split()
        
        
        print(f'Language: {language}')
        print('-----------------------')
        print(f'Number of sentences\t:\t {number_of_sentences}')
        print(f'Number of words\t\t:\t {number_of_words}')
        print(f'Number of unique words\t:\t {number_of_unique_words}')
        print(f'Sample extract\t\t:\t {sample_extract}...\n')

In [22]:
show_statistics(raw_data)

Language: sk
-----------------------
Number of sentences	:	 100
Number of words		:	 100
Number of unique words	:	 100
Sample extract		:	 ...

Language: en
-----------------------
Number of sentences	:	 100
Number of words		:	 100
Number of unique words	:	 100
Sample extract		:	 ...

Language: ch
-----------------------
Number of sentences	:	 10
Number of words		:	 10
Number of unique words	:	 10
Sample extract		:	 ...

